### Notebook for the following paper:

http://iopscience.iop.org/article/10.3847/1538-3881/aa9ab8/meta

* Pavlos, Karim, and Nicolás Castro

###### General Overview:
The paper is on treating features as distributions.  It uses Gaussian Processes (GP) to treat the time series like a distribution, making use of both the measurement error given in the data, $\epsilon$, and the error associated with gaps in the time series.  The paper also covers sampling from the distributions and training a random forest based on the samples (randomness in the data from the sampling, not the feature selection).